In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# generate 3D data
x = np.arange(-1, 1, 0.01)
y = np.arange(-1, 1, 0.01)
xx, yy = np.meshgrid(x, y)
zz = (0.4 ** 2 - (0.6 - (xx ** 2 + yy ** 2) ** 0.5) ** 2) ** 0.5

In [ ]:
# make a 3D plot
from <...> import <...>

# create 3D axes
fig = plt.<...>(figsize=(7, 7))
ax = plt.<...>(projection='3d')

# use pyplot contour function
ax.contour(<...>, yy, <...>, 30)

# remove axis
ax.axis(<...>)

plt.show()

In [ ]:
# import seaborn
import <...> as <...>

# create a dataframe, converting the 2D arrays to 1D using np.ravel
data = <...>({
    'x': <...>(xx),
    'y': <...>(yy),
    'z': <...>(zz),
})

# create a new column w
<...> =  np.sin(10 * (data['x'] ** 2 + data['y'] ** 2)) / 10

# add random noise to w column
data['w'] = data['w'].apply(lambda x: <...> + np.<...>.<...>() * 0.02)

# plot using sns.relplot
<...>(data=data, x='z', y='w', hue='y')

In [ ]:
# make another 3D plot
fig = plt.figure(figsize=(7, 7))
ax = plt.axes(projection='3d')

# use scatter to plot data this time, using matplotlib, give transparency of 0.1
# using the alpha keyword argument
ax.<...>(data['x'], data['y'], data['w'], c=data['w'], <...>=0.1)

# set all the limits to [-1, 1]
ax.<...>(-1, 1)
ax.<...>(-1, 1)
ax.<...>(-1, 1)
plt.show()

In [ ]:
# plot z vs x, using blue dots with transparency of 0.01
plt.plot(data['x'], data['z'], <...>, <...>)

# do the same for w vs y but with red dots
plt.plot(data['y'], data['w'], <...>, <...>)

In [ ]:
# use sns displot to plot a contour map of of w vs x
<...>(data=<...>, x=<...>, <...>=<...>, kind='kde')

In [ ]:
# In this section we will revisit the use of classes - making two classes a Component class and a Signal class
# The result will be the ability to sum up many sine and cosine waves, and then extract their features and plot their trace

from typing import List
import scipy.fft

# create signal component class
class Component:
    def <...>(
        self, 
        coefficient: float, 
        frequency: float, 
        phase: float = 0, 
        offset: float = 0, 
        noise: float = 0, 
        kind='cos'
    ):
        "A sine or cosine wave"
        
        # set attributes
        self.coefficient = coefficient
        self.frequency = frequency
        self.phase = phase
        self.offset = offset
        self.noise = noise
        self.kind = kind
        
        if kind == 'cos':
            self.function = <...>
            
        <...> kind == <...>:
            <...> = np.sin
            
        else:
            <...> TypeError(f"The keyword `kind` should either be 'cos' or 'sin', you have put {kind}!")
    
    # make __call__ method to generate the signal
    def __call__(self, x: np.ndarray) -> np.ndarray:
        signal = self.<...> * self.<...>(self.frequency * x + self.phase) + self.<...>
        signal = [i + self.<...> * np.random.random() for i in <...>]
        return <...>
    
# Create a sine component
class Sine(Component):
    def __init__(self, *args, **kwargs):
        <...>.__init__(kind='sin', *args, **kwargs)

# create a cosine component
class Cosine(<...>):
    def __init__(<...>, *args, **kwargs):
        super().<...>(kind='cos', *args, **kwargs)

# signal generation class
class Signal:
    def __init__(self, components: List[Component]):
        """A contained for a list of components"""
        if isinstance(components, list) and len(components) != 0:
            self.components = components
            
        else:
            self.components = []
    
    # Do call in the same way as before
    def __call__(self, x: np.ndarray) -> np.ndarray:
        # get result of first component
        result = self.components[0](x)
        
        # loop over the rest
        for component in <...>[1:]:
            result = np.sum([result, <...>], axis=<...>)
        
        # return result
        <...> result
    
    # add functionality to plot signal
    def plot(self, x: np.ndarray, ax: plt.Axes = None, plot_components: bool = False):
        if not ax:
            # create axes
            <...>, <...> = plt.<...>(figsize=(10, 5))
            ax.<...>('Time')
            ax.<...>('Signal')
            
        # call self
        signal = self.__call__(<...>)
        
        # plot signal on axes
        <...>.plot(x, <...>, 'k-')
        
        # ADVANCED: make option for plotting the individual components
        if plot_components:
            for component in self.components:
                <...>
    
    # get fourier transform
    def fourier(self, x: np.ndarray) -> List[np.ndarray]:
        signal = <...>
        N = len(x)
        delta = <...>
        # create the frequency axis
        xt = np.linspace(0.0, 1.0 / (2.0 * delta), N // 2)
        
        # do fourier transform and normalise
        yt = <...>
        yt = 2.0/N * np.abs(yt[0:N//2])
        
        return xt, yt


In [ ]:
### - TO TEST THE CLASSES, RUN THIS CODE AND SEE WHAT HAPPENS
### - WHEN YOU HAVE SOMETHING THAT WORKS, EXPERIMENT WITH DIFFERENT SIGNAL COMBINATIONS

# Generate signal
signal = Signal([
    Cosine(1, 440, noise=5.0),
    Sine(2, 700, offset=-0.5),
    Sine(2, 350, offset=-10),
    Sine(1, 30, phase=0.1),
    Cosine(2, 200, offset=0.5),
    Sine(5, 3300, phase=0.1, noise=5.0),
    Sine(0.5, 20, phase=-0.1, noise=0.2)
])

# plot signal
signal.plot(np.arange(0, 0.5, 0.001), plot_components=True)

# plot fourier transform
xt, yt = signal.fourier(np.arange(0, 0.5, 0.001))
fig, ax = plt.subplots()
ax.plot(xt, yt)
ax.set_xlabel('Frequency')
ax.set_ylabel('Amplitude')
plt.show()